In [1]:
import os
os.chdir("../")
%pwd

'c:\\Code Playground\\Programs_github\\Chicken-Disease-Classification-project'

In [9]:
# change the config.yaml file
# initialize the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
# Update configuration manager
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [5]:
# Create new component

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e


[2023-07-30 15:12:25,552: INFO: common] yaml file: config\config.yaml loaded successfully
[2023-07-30 15:12:25,555: INFO: common] yaml file: params.yaml loaded successfully
[2023-07-30 15:12:25,556: INFO: common] directory created at: artifacts
[2023-07-30 15:12:25,559: INFO: common] directory created at: artifacts\prepare_callbacks\checkpoint_dir
[2023-07-30 15:12:25,562: INFO: common] directory created at: artifacts\prepare_callbacks\tensorboard_log_dir


In [8]:
tf.__version__

'2.13.0'

In [15]:
import time
timestamp = time.strftime("%Y%m%d-%H%M%S")
f"tb_logs_at{timestamp}"

'tb_logs_at20230730-152809'